<center>
<img src="https://cordoba.tur.ar/wp-content/uploads/2020/04/cropped-logo-Gobierno.png" width="600" height="600" />
 </center>       

<center>
<h3>Secretaría de Transporte de la Provincia de Córdoba<h3>   <h1>Análisis Funcionamiento Boletos de Gratuidad BEG-BOS-BAM<h1>  <h0>Interurbanos<h0>
</center>

#### Importación de Librerías a Utilizar

In [1]:
import io
import os
import pandas as pd
import numpy as np
import seaborn as sbn
import xlsxwriter
import openpyxl

%matplotlib inline
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows", None)

#### Lectura Tablas Interurbano

En este paso se leen los archivos correspondientes a las tablas de movimientos de usuarios de Interurbanos. Se define una ruta del archivo en la primer línea (folder) y se listan los archivos (filelist). El código está escrito para traer todos los archivos dentro de la carpeta que se le indicó al principio. Esto se puede modificar según como nombren a los archivos. Además, en caso de no querer llamar a la lista completa de archivos, se puede definir a mano esa filelist indicando el nombre de los archivos que se quieren leer. **Para identificar las lineas correspondientes a cada programa, se creó una columna nueva 'PROGRAMA' que toma el nombre del archivo como identificador**

### Opcion A
Sirve para unir los csv de BEG interurbano por el peso

In [ ]:
#Veo que archivos tengo en esa carpeta
import os
import pandas as pd
#read the path
file_path = "C:/Users/Control 2/Documents/ely python/Noviembre/Interurbano/"
#list all the files from the directory
file_list = os.listdir(file_path)
file_list

In [ ]:
#read the csv file into a dataframe
df1 = pd.read_csv("C:/Users/Control 2/Documents/ely python/Noviembre/Interurbano/BEG/BEG 2022 Noviembre Interurbano part 1.csv")

In [ ]:
df2 = pd.read_csv("C:/Users/Control 2/Documents/ely python/Noviembre/Interurbano/BEG/BEG 2022 Noviembre Interurbano part 2.csv")

In [ ]:
df = pd.concat([df1, df2], ignore_index=True, sort=False)

In [ ]:
df['PROGRAMA'] = 'BEG'

In [ ]:
df.head(5)

### Opcion B
Esta opcion no sirve para unir los BEG porque piensan que son objetos distintos, pero sirve para concatenar el BAM y el BOS

In [2]:
"""Se define ruta de los archivos"""
folder = 'C:/Users/Control 2/Documents/ely python/Noviembre/Interurbano/'


"""Se genera la lista de todos los archivos con las dos opciones, lista entera o manual"""
filelist = [file for file in os.listdir(folder) if file.endswith('BEG 2022 Mayo Interurbano.csv')]
filelist = [file for file in os.listdir(folder)]

dataframe = []

for file in filelist:
    try:
        df = pd.read_csv(folder + str(file), error_bad_lines=False, warn_bad_lines=False)
        
        df['file'] = str(file)
        df['PROGRAMA'] = df['file'].str[0:3]
        df = df.drop(columns='file')
        dataframe.append(df)
    except OSError:
        pass

df = pd.concat(dataframe, axis=0)

C:\Users\Control 2\AppData\Local\Temp\ipykernel_5092\55097586.py:13: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(folder + str(file), error_bad_lines=False, warn_bad_lines=False)
C:\Users\Control 2\AppData\Local\Temp\ipykernel_5092\55097586.py:13: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(folder + str(file), error_bad_lines=False, warn_bad_lines=False)
C:\Users\Control 2\AppData\Local\Temp\ipykernel_5092\55097586.py:13: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(folder + str(file), error_bad_lines=False, warn_bad_lines=False)
C:\Users\Control 2\AppData\Local\Temp\ipykernel_5092\55097586.py:13: FutureWarning: The warn_bad_lines argument has been depr

In [ ]:
#df = 'C:/Users/usuario/Desktop/Secretaría Tpte - 2021/Boletos - Python/Movimientos boletos 2022/Interurbano/BEG/BEG 2022 Abril Interurbano.xlsx'

In [3]:
df.columns

Index(['ID_AUTORIZACION', 'CUIL', 'ID_ORIGEN', 'ID_DESTINO', 'ID_EMPRESA',
       'INICIO_FRANJA_HORARIA', 'FINAL_FRANJA_HORARIA', 'CANT_USO_X_DIA',
       'CANT_USO_X_MES', 'CANT_USO_X_DIA_SEM', 'CANT_USO_X_SEMANA',
       'ID_TIPO_SOLICITANTE', 'VIAJA_FERIADO_SN', 'FEC_DESDE', 'FEC_HASTA',
       'ID_NIVEL_EDUCATIVO', 'EXCEP', 'ID_DISTANCIA', 'ID_MOVIMIENTO',
       'FEC_MOVIMIENTO', 'ID_ORIGEN.1', 'ID_DESTINO.1', 'ID_PROGRAMA',
       'UID_TARJETA', 'LATITUD', 'LONGITUD', 'NRO_INTERNO',
       'ID_ADMINISTRADORA', 'VALIDADO_SN', 'VALORIZAR_SN', 'FEC_LIQUIDACION',
       'LIQUIDADO_SN', 'CUIT_TRANSPORTISTA', 'PRECIO_GOBIERNO',
       'PRECIO_VALORIZADO', 'ID_RESULTADO', 'ID_EMPRESA.1', 'FEC_INGRESO',
       'MONTO_ULTIMO', 'PROGRAMA'],
      dtype='object')

#### Lectura Tablas Referencias

Además del input principal que es la tabla de movimientos. Para poder trabajar se necesita cargar el archivo que contiene las referencias para las variables codificadas. El procedimiento es el mismo. Se define una ruta para el achivo y luego se lo llama. En este caso, se generaron a partir del mismo archivo, tres tablas distintas cada una conteniendo información de una hoja distinta del archivo excel.

In [4]:
path = 'C:/Users/Control 2/Dropbox/TRANSPORTE/BOLETOS/Python Scripts/Referencias.xlsx'

tipo_solicitante = pd.read_excel(path, sheet_name = 'TIPO_SOLICITANTE', engine='openpyxl')
nivel_educativo = pd.read_excel(path, sheet_name = 'NIVEL_EDUCATIVO', engine='openpyxl')
empresas = pd.read_excel(path, sheet_name = 'EMPRESAS', engine='openpyxl')

#### Definición Archivo donde se guardarán todos los resultados obtenidos en esta notebook

Antes de comenzar el análisis, creamos un archivo excel vacío donde iremos guardando cada uno de los resultados obtenidos en los siguientes pasos. Acá nuevamente es necesario indicar una ruta y colocarle un nombre al archivo.

In [5]:
writer = pd.ExcelWriter('C:/Users/Control 2/Documents/ely python/Noviembre/MOV_INTER_Noviembre_2022_BAMBOS.xlsx', engine='xlsxwriter')

#### Cambio Formato Fecha

El formato con el que viene la fecha originalmente en la tabla no es conveniente para trabajar por lo que generamos variables nuevas que indiquen el dia, mes y año del movimiento.

In [6]:
base_inter = df

In [7]:
base_inter = df
base_inter['DIA_MOVIMIENTO'] = base_inter['FEC_MOVIMIENTO'].str[0:10]
base_inter['ANO_MES_MOVIMIENTO'] = base_inter['FEC_MOVIMIENTO'].str[3:10]
base_inter['ANO_MOVIMIENTO'] = base_inter['FEC_MOVIMIENTO'].str[6:10]
base_inter.head(5)

,ID_AUTORIZACION,CUIL,ID_ORIGEN,ID_DESTINO,ID_EMPRESA,INICIO_FRANJA_HORARIA,FINAL_FRANJA_HORARIA,CANT_USO_X_DIA,CANT_USO_X_MES,CANT_USO_X_DIA_SEM,CANT_USO_X_SEMANA,ID_TIPO_SOLICITANTE,VIAJA_FERIADO_SN,FEC_DESDE,FEC_HASTA,ID_NIVEL_EDUCATIVO,EXCEP,ID_DISTANCIA,ID_MOVIMIENTO,FEC_MOVIMIENTO,ID_ORIGEN.1,ID_DESTINO.1,ID_PROGRAMA,UID_TARJETA,LATITUD,LONGITUD,NRO_INTERNO,ID_ADMINISTRADORA,VALIDADO_SN,VALORIZAR_SN,FEC_LIQUIDACION,LIQUIDADO_SN,CUIT_TRANSPORTISTA,PRECIO_GOBIERNO,PRECIO_VALORIZADO,ID_RESULTADO,ID_EMPRESA.1,FEC_INGRESO,MONTO_ULTIMO,PROGRAMA,DIA_MOVIMIENTO,ANO_MES_MOVIMIENTO,ANO_MOVIMIENTO
0,4328075,27027567237,1092.0,1343.0,673,00:00,23:59,2,0,0,0,8,S,12/09/2019 09:13:41 a.m.,19/09/2019,NaN,NaN,6.0,154828600,11/11/2022 01:14:08 p.m.,1092,1343,3,4987AD76,30.980131,64.094316,690,1,S,S,NaN,NaN,30678714344,NaN,"214,9125",3,673,12/11/2022 12:00:51 a.m.,"9891,92",BAM,11/11/2022,11/2022,2022
1,4695462,20084531155,1248.0,1.0,677,00:00,23:59,2,0,0,0,8,S,26/10/2019 12:18:32 p.m.,02/11/2019,NaN,NaN,6.0,154906161,14/11/2022 05:02:28 p.m.,1,201,3,69DABF76,31.425068,64.172750,5100,1,S,S,NaN,NaN,30707788832,NaN,"287,6475",3,677,15/11/2022 12:46:00 a.m.,"8992,37",BAM,14/11/2022,11/2022,2022
2,4852194,27100483667,1.0,1445.0,677,00:00,23:59,2,0,0,0,8,S,13/11/2019 12:30:10 p.m.,20/11/2019,NaN,NaN,6.0,155270018,21/11/2022 07:22:30 p.m.,1,1445,3,29A4C076,31.350008,64.176090,5239,1,S,S,NaN,NaN,30707788832,NaN,"168,8625",3,677,22/11/2022 12:14:06 a.m.,"13131,31",BAM,21/11/2022,11/2022,2022
3,4958275,20079767213,1248.0,1.0,677,00:00,23:59,2,0,0,0,8,S,27/11/2019 11:08:20 a.m.,04/12/2019,NaN,NaN,6.0,154518780,09/11/2022 06:52:46 p.m.,1,1248,3,99CBEB33,31.424975,64.172483,5117,1,S,S,NaN,NaN,30707788832,NaN,"287,6475",3,677,10/11/2022 02:10:54 p.m.,"8757,48",BAM,09/11/2022,11/2022,2022
4,5208916,27101141492,6103.0,1498.0,677,00:00,23:59,2,0,0,0,8,S,27/12/2019 01:35:00 p.m.,03/01/2020,NaN,NaN,6.0,155623546,26/11/2022 03:26:31 p.m.,1498,1092,3,79DB7C1D,31.274548,64.169763,5223,1,S,N,NaN,NaN,30707788832,NaN,NaN,3,677,27/11/2022 12:12:22 a.m.,"10100,00",BAM,26/11/2022,11/2022,2022


## Veo como es el archivo

In [8]:
base_inter.shape

(52862, 43)

In [9]:
pd.value_counts (base_inter['ID_DISTANCIA'])

6.0    19947
4.0    19377
7.0    13135
5.0      146
Name: ID_DISTANCIA, dtype: int64

In [10]:
pd.value_counts (base_inter['ID_RESULTADO'])

1    50955
3     1907
Name: ID_RESULTADO, dtype: int64

In [11]:
base_inter['LIQUIDADO_SN'].value_counts()

S    50843
Name: LIQUIDADO_SN, dtype: int64

##### Veo los movimientos por dia para ver como voy
Recordar los dias que no aparezcan es porque fue feriado, ojo si no es feriado preguntar a Maxi, si es paro tampoco hay

In [12]:
#base_inter.groupby(['DIA_MOVIMIENTO', 'PROGRAMA']).agg({'ID_MOVIMIENTO':'count'}).reset_index()

cuales son los rechazon

In [13]:
#table7 = pd.pivot_table(base_inter, values ='ID_MOVIMIENTO', index =['ID_EMPRESA'],
#                         columns =['ID_DISTANCIA'], aggfunc = 'count')
#table7

In [14]:
#table7.plot(kind='bar', figsize=(25, 10)).set_title('Cantidad de movimientos por Empresa y Distancia \n', fontweight='bold', fontsize=16)
#plt.ylabel('Cantidad de autorizaciones', fontsize=14)

## BUSCO UN SERVICIO EN ESPECIAL

In [15]:
#cole = base_inter.loc[base_inter['ID_EMPRESA']== 679]
#cole.head(10)

#### Formato Distancia - Agrega Columna "TIPO_INTER"

In [16]:
base_inter.shape

(52862, 43)

In [17]:
#string indices must be integersdic = {1: 'Urbanos', 2:'Interurbano menos de 100km', 3:'Interurbano mas de 100km', 4:'Interurbano menos de 100km', 6:'Interurbano menos de 60km', 7:'Interurbano mas de 60km'}
#base_inter['TIPO_INTER'] = base_inter['ID_DISTANCIA'].map(dic)
#base_inter

## Creo la clasificación de + 100 km - 100km

In [18]:
base_inter['TIPO_INTER'] = np.where(base_inter['ID_DISTANCIA']==2, 'Interurbano menos de 100km', 'Interurbano más de 100km')
base_inter['TIPO_INTER_TRAMOS'] = np.where(base_inter['ID_DISTANCIA']<=100, 'Interurbano menos de 100km', 'Interurbano más de 100km')
base_inter.head(5)

,ID_AUTORIZACION,CUIL,ID_ORIGEN,ID_DESTINO,ID_EMPRESA,INICIO_FRANJA_HORARIA,FINAL_FRANJA_HORARIA,CANT_USO_X_DIA,CANT_USO_X_MES,CANT_USO_X_DIA_SEM,CANT_USO_X_SEMANA,ID_TIPO_SOLICITANTE,VIAJA_FERIADO_SN,FEC_DESDE,FEC_HASTA,ID_NIVEL_EDUCATIVO,EXCEP,ID_DISTANCIA,ID_MOVIMIENTO,FEC_MOVIMIENTO,ID_ORIGEN.1,ID_DESTINO.1,ID_PROGRAMA,UID_TARJETA,LATITUD,LONGITUD,NRO_INTERNO,ID_ADMINISTRADORA,VALIDADO_SN,VALORIZAR_SN,FEC_LIQUIDACION,LIQUIDADO_SN,CUIT_TRANSPORTISTA,PRECIO_GOBIERNO,PRECIO_VALORIZADO,ID_RESULTADO,ID_EMPRESA.1,FEC_INGRESO,MONTO_ULTIMO,PROGRAMA,DIA_MOVIMIENTO,ANO_MES_MOVIMIENTO,ANO_MOVIMIENTO,TIPO_INTER,TIPO_INTER_TRAMOS
0,4328075,27027567237,1092.0,1343.0,673,00:00,23:59,2,0,0,0,8,S,12/09/2019 09:13:41 a.m.,19/09/2019,NaN,NaN,6.0,154828600,11/11/2022 01:14:08 p.m.,1092,1343,3,4987AD76,30.980131,64.094316,690,1,S,S,NaN,NaN,30678714344,NaN,"214,9125",3,673,12/11/2022 12:00:51 a.m.,"9891,92",BAM,11/11/2022,11/2022,2022,Interurbano más de 100km,Interurbano menos de 100km
1,4695462,20084531155,1248.0,1.0,677,00:00,23:59,2,0,0,0,8,S,26/10/2019 12:18:32 p.m.,02/11/2019,NaN,NaN,6.0,154906161,14/11/2022 05:02:28 p.m.,1,201,3,69DABF76,31.425068,64.172750,5100,1,S,S,NaN,NaN,30707788832,NaN,"287,6475",3,677,15/11/2022 12:46:00 a.m.,"8992,37",BAM,14/11/2022,11/2022,2022,Interurbano más de 100km,Interurbano menos de 100km
2,4852194,27100483667,1.0,1445.0,677,00:00,23:59,2,0,0,0,8,S,13/11/2019 12:30:10 p.m.,20/11/2019,NaN,NaN,6.0,155270018,21/11/2022 07:22:30 p.m.,1,1445,3,29A4C076,31.350008,64.176090,5239,1,S,S,NaN,NaN,30707788832,NaN,"168,8625",3,677,22/11/2022 12:14:06 a.m.,"13131,31",BAM,21/11/2022,11/2022,2022,Interurbano más de 100km,Interurbano menos de 100km
3,4958275,20079767213,1248.0,1.0,677,00:00,23:59,2,0,0,0,8,S,27/11/2019 11:08:20 a.m.,04/12/2019,NaN,NaN,6.0,154518780,09/11/2022 06:52:46 p.m.,1,1248,3,99CBEB33,31.424975,64.172483,5117,1,S,S,NaN,NaN,30707788832,NaN,"287,6475",3,677,10/11/2022 02:10:54 p.m.,"8757,48",BAM,09/11/2022,11/2022,2022,Interurbano más de 100km,Interurbano menos de 100km
4,5208916,27101141492,6103.0,1498.0,677,00:00,23:59,2,0,0,0,8,S,27/12/2019 01:35:00 p.m.,03/01/2020,NaN,NaN,6.0,155623546,26/11/2022 03:26:31 p.m.,1498,1092,3,79DB7C1D,31.274548,64.169763,5223,1,S,N,NaN,NaN,30707788832,NaN,NaN,3,677,27/11/2022 12:12:22 a.m.,"10100,00",BAM,26/11/2022,11/2022,2022,Interurbano más de 100km,Interurbano menos de 100km


In [19]:
Antes=base_inter

In [20]:
pd.value_counts (base_inter['TIPO_INTER'])

Interurbano más de 100km    52862
Name: TIPO_INTER, dtype: int64

## Filtra Errores y usos no válidos

Hay movimientos con ID DISTANCIA = 1 esos son viajes de Urbanos. Se eliminan antes de seguir.

In [21]:
base_inter = base_inter[base_inter['ID_DISTANCIA']!=1]

lista_empresas = [4, 12, 14, 17, 687]

#base_inter= base_inter[['ID_EMPRESA','ID_EMPRESA.1', 'ID_RESULTADO']]
#CHEQUEAR QUE SOLO HAYA ESTOS CASOS EN BEG !!!!!
base_inter['FILTER'] = np.where(base_inter['ID_EMPRESA.1'].isin(lista_empresas)&(base_inter['PROGRAMA']=='BEG')&(base_inter['ID_RESULTADO']!=1),'DROP', np.nan)

#creo una ultima columna que la veo con la clasificacion nan y DROP
base_inter.head(3)


,ID_AUTORIZACION,CUIL,ID_ORIGEN,ID_DESTINO,ID_EMPRESA,INICIO_FRANJA_HORARIA,FINAL_FRANJA_HORARIA,CANT_USO_X_DIA,CANT_USO_X_MES,CANT_USO_X_DIA_SEM,CANT_USO_X_SEMANA,ID_TIPO_SOLICITANTE,VIAJA_FERIADO_SN,FEC_DESDE,FEC_HASTA,ID_NIVEL_EDUCATIVO,EXCEP,ID_DISTANCIA,ID_MOVIMIENTO,FEC_MOVIMIENTO,ID_ORIGEN.1,ID_DESTINO.1,ID_PROGRAMA,UID_TARJETA,LATITUD,LONGITUD,NRO_INTERNO,ID_ADMINISTRADORA,VALIDADO_SN,VALORIZAR_SN,FEC_LIQUIDACION,LIQUIDADO_SN,CUIT_TRANSPORTISTA,PRECIO_GOBIERNO,PRECIO_VALORIZADO,ID_RESULTADO,ID_EMPRESA.1,FEC_INGRESO,MONTO_ULTIMO,PROGRAMA,DIA_MOVIMIENTO,ANO_MES_MOVIMIENTO,ANO_MOVIMIENTO,TIPO_INTER,TIPO_INTER_TRAMOS,FILTER
0,4328075,27027567237,1092.0,1343.0,673,00:00,23:59,2,0,0,0,8,S,12/09/2019 09:13:41 a.m.,19/09/2019,NaN,NaN,6.0,154828600,11/11/2022 01:14:08 p.m.,1092,1343,3,4987AD76,30.980131,64.094316,690,1,S,S,NaN,NaN,30678714344,NaN,"214,9125",3,673,12/11/2022 12:00:51 a.m.,"9891,92",BAM,11/11/2022,11/2022,2022,Interurbano más de 100km,Interurbano menos de 100km,nan
1,4695462,20084531155,1248.0,1.0,677,00:00,23:59,2,0,0,0,8,S,26/10/2019 12:18:32 p.m.,02/11/2019,NaN,NaN,6.0,154906161,14/11/2022 05:02:28 p.m.,1,201,3,69DABF76,31.425068,64.172750,5100,1,S,S,NaN,NaN,30707788832,NaN,"287,6475",3,677,15/11/2022 12:46:00 a.m.,"8992,37",BAM,14/11/2022,11/2022,2022,Interurbano más de 100km,Interurbano menos de 100km,nan
2,4852194,27100483667,1.0,1445.0,677,00:00,23:59,2,0,0,0,8,S,13/11/2019 12:30:10 p.m.,20/11/2019,NaN,NaN,6.0,155270018,21/11/2022 07:22:30 p.m.,1,1445,3,29A4C076,31.350008,64.176090,5239,1,S,S,NaN,NaN,30707788832,NaN,"168,8625",3,677,22/11/2022 12:14:06 a.m.,"13131,31",BAM,21/11/2022,11/2022,2022,Interurbano más de 100km,Interurbano menos de 100km,nan


In [22]:
#veo cuantas son las drop, las voy a sacar del analisis
pd.value_counts (base_inter['FILTER'])

nan    52862
Name: FILTER, dtype: int64

In [23]:
#los elimino
base_inter = base_inter[base_inter['FILTER']!='DROP']
base_inter.head(3)

,ID_AUTORIZACION,CUIL,ID_ORIGEN,ID_DESTINO,ID_EMPRESA,INICIO_FRANJA_HORARIA,FINAL_FRANJA_HORARIA,CANT_USO_X_DIA,CANT_USO_X_MES,CANT_USO_X_DIA_SEM,CANT_USO_X_SEMANA,ID_TIPO_SOLICITANTE,VIAJA_FERIADO_SN,FEC_DESDE,FEC_HASTA,ID_NIVEL_EDUCATIVO,EXCEP,ID_DISTANCIA,ID_MOVIMIENTO,FEC_MOVIMIENTO,ID_ORIGEN.1,ID_DESTINO.1,ID_PROGRAMA,UID_TARJETA,LATITUD,LONGITUD,NRO_INTERNO,ID_ADMINISTRADORA,VALIDADO_SN,VALORIZAR_SN,FEC_LIQUIDACION,LIQUIDADO_SN,CUIT_TRANSPORTISTA,PRECIO_GOBIERNO,PRECIO_VALORIZADO,ID_RESULTADO,ID_EMPRESA.1,FEC_INGRESO,MONTO_ULTIMO,PROGRAMA,DIA_MOVIMIENTO,ANO_MES_MOVIMIENTO,ANO_MOVIMIENTO,TIPO_INTER,TIPO_INTER_TRAMOS,FILTER
0,4328075,27027567237,1092.0,1343.0,673,00:00,23:59,2,0,0,0,8,S,12/09/2019 09:13:41 a.m.,19/09/2019,NaN,NaN,6.0,154828600,11/11/2022 01:14:08 p.m.,1092,1343,3,4987AD76,30.980131,64.094316,690,1,S,S,NaN,NaN,30678714344,NaN,"214,9125",3,673,12/11/2022 12:00:51 a.m.,"9891,92",BAM,11/11/2022,11/2022,2022,Interurbano más de 100km,Interurbano menos de 100km,nan
1,4695462,20084531155,1248.0,1.0,677,00:00,23:59,2,0,0,0,8,S,26/10/2019 12:18:32 p.m.,02/11/2019,NaN,NaN,6.0,154906161,14/11/2022 05:02:28 p.m.,1,201,3,69DABF76,31.425068,64.172750,5100,1,S,S,NaN,NaN,30707788832,NaN,"287,6475",3,677,15/11/2022 12:46:00 a.m.,"8992,37",BAM,14/11/2022,11/2022,2022,Interurbano más de 100km,Interurbano menos de 100km,nan
2,4852194,27100483667,1.0,1445.0,677,00:00,23:59,2,0,0,0,8,S,13/11/2019 12:30:10 p.m.,20/11/2019,NaN,NaN,6.0,155270018,21/11/2022 07:22:30 p.m.,1,1445,3,29A4C076,31.350008,64.176090,5239,1,S,S,NaN,NaN,30707788832,NaN,"168,8625",3,677,22/11/2022 12:14:06 a.m.,"13131,31",BAM,21/11/2022,11/2022,2022,Interurbano más de 100km,Interurbano menos de 100km,nan


In [24]:
base_inter.shape

(52862, 46)

###### RECORDATORIO
---
Para pasar a episodios eliminados tengo que sumar DROP + pd.value_counts (base_inter['ID_DISTANCIA']) = 2 y 3 

### Mapeo con el diccionario
---

Hago un mapeo con un diccionario para que me traiga el nombre de las empresas, para ver cuales son en las que pasa este problema

In [25]:
Diccionario_Empresas = {1 : 'COATA S.A.',
2 : 'COTA LA CALERA  LTDA.',
4 : 'COLONIA TIROLESA S.R.L.',
5 : 'EMPRESA CORDOBA S.R.L.',
6 : 'EL PORVENIR S.R.L.',
8 : 'EXPRESO ACHIRAS S.R.L.',
10 : 'LA ESTRELLA S.R.L.',
12 : 'MALVINAS ARGENTINAS S.R.L.',
13 : 'MAR CHIQUITA S.R.L.',
14 : 'EL QUEBRACHAL S.R.L.',
17 : 'SIERRAS DE CALAMUCHITA S.R.L.',
18 : 'EMP TRANSPORTE MORTEROS S.R.L.',
21 : 'S.A. TRANSPORTE CIUDAD DE RIO CUARTO[Inter]',
25 : 'EMRPESA SARMIENTO S.R.L.[Inter]',
26 : 'RIO BER S.R.L.',
29 : 'EXPRESO DIFERENCIAL CORDOBA RIO CUARTO S.R.L.',
31 : 'PAJARO BLANCO S.R.L.',
33 : 'EXPRESO VILLA DEL ROSARIO S.A.',
34 : 'CIRCUNVALACION S.A.',
35 : 'INTERCORDOBA S.A.',
36 : 'LUMASA VIAJES S.R.L.',
37 : 'LA SERRANITA S.R.L.',
38 : 'CAR-BUS S.R.L.',
39 : 'EMPRESA EL TURISTA S.R.L.',
40 : 'ALEJANDRO S.R.L.',
41 : 'SOCSA S.R.L.',
45 : 'EXCURSIONES SIERRAS DE CALAMUCHITA S.R.L.',
46 : 'SIERRA BUS S.R.L.',
48 : 'EMPRENDIMIENTOS S.R.L. (FONO BUS)',
49 : 'CAR-COR S.R.L.',
50 : 'DIFERENCIAL TRANSIERRAS S.R.L.',
51 : 'PANAHOLMA S.R.L.',
52 : 'TRANSPORTE DEAN FUNES S.R.L.',
53 : 'EL TATU CARRETERO S.R.L.',
54 : 'LA DILIGENCIA VIP S.R.L.',
60 : 'CANELLO HNOS S.R.L.',
62 : 'EL MILAGRO S.R.L.',
501 : 'CONIFERAL SACIF',
502 : 'TAMSE',
603 : 'CHIACCHIERA HENRI LUIS [JESÚS MARÍA-COLONIA CAROYA]',
605 : 'EMPRESA MARTINEZ S.R.L.[Alta Gracia]',
606 : 'TRANSPORTE AUTOMOTOR GARAY S.R.L',
607 : 'GOOD TRAVEL TOURS S.R.L.[ALTA GRACIA]',
610 : 'LUMASA SA[LA FALDA]',
611 : 'MEICHTRI EDUARD RICARDO',
612 : 'MUN CDAD DE S FRANCISCO',
613 : 'RIO BUS S.R.L.[RIO TERCERO]',
615 : 'TRANS BUS S.R.L.[VILLA MARIA]',
616 : 'TRANSPORTE AUTOMOTOR GARAY S.R.L [ALTA GRACIA]',
619 : 'EMRPESA SARMIENTO S.R.L.[COSQUIN]',
620 : 'MA - COR S.R.L.[MALAGUEÑO]',
622 : 'INTERCORDOBA S.A.[RÍO CEBALLOS]',
623 : 'S.A. TRANSPORTE CIUDAD DE RIO CUARTO [RIO CUARTO]',
624 : 'EXPRESO DIFERENCIAL CORDOBA RIO CUARTO S.R.L.',
625 : 'EL TATU CARRETERO S.R.L.',
626 : 'EMRPESA SARMIENTO S.R.L.',
627 : 'EXCURSIONES SIERRAS DE CALAMUCHITA S.R.L.',
628 : 'INTERCORDOBA S.A.',
629 : 'LUMASA VIAJES S.R.L.',
630 : 'SIERRA BUS S.R.L.',
632 : 'CANELLO HNOS S.R.L.',
633 : 'CAR-COR S.R.L.',
634 : 'DIFERENCIAL TRANSIERRAS S.R.L.',
635 : 'EDER SERVICIO DIFERENCIAL S.R.L.',
636 : 'EL MILAGRO S.R.L.',
637 : 'EMPRENDIMIENTOS S.R.L. (FONO BUS)[INTER]',
638 : 'LA DILIGENCIA VIP S.R.L.',
639 : 'MA - COR S.R.L.',
640 : 'PANAHOLMA S.R.L.',
641 : 'TRANSPORTE DEAN FUNES S.R.L.',
642 : 'TRANSPORTE JAMES CRAIK S.R.L.',
646 : 'ZETA BUS S.R.L.',
652 : 'EMPRENDIMIENTOS S.R.L. (FONO BUS)[TANTI]',
653 : 'PAJARO BLANCO S.R.L.[SANTA ROSA DE CALAMUCHITA]',
656 : 'EMPRENDIMIENTOS S.R.L. (FONO BUS)[MENDIOLAZA]',
657 : 'EMPRENDIMIENTOS S.R.L. (FONO BUS)[SALDAN]',
658 : 'EMPRENDIMIENTOS S.R.L. (FONO BUS)[BIALET MASSE]',
659 : 'EMPRENDIMIENTOS S.R.L. (FONO BUS)[LA GRANJA]',
660 : 'EMPRENDIMIENTOS S.R.L. (FONO BUS)[SANTA MARIA DE PUNILLA]',
671 : 'ERSA URBANO S.A.',
672 : 'AUTOBUSES SANTA FE S.R.L.',
673 : 'EMPRENDIMIENTOS S.R.L. (FONO BUS)[INTER]',
674 : 'EXPRESO DIFERENCIAL CORDOBA RIO CUARTO S.R.L.[EMBALSE]',
675 : 'SI! BUS EMPRESA SARMIENTO S.R.L. - INTERCORDOBA S.A. UNION TRANSITORIA DE EMPRESAS[VILLA ALLENDE]',
676 : 'CANELLO HNOS. S.R.L. [DESPEÑADEROS]',                      
677 : 'ERSA URBANO S.A.[Inter]',
680 : 'EXPRESO DIFERENCIAL CORDOBA RIO CUARTO S.R.L.[VILLA GENERAL BELGRANO (INTERCOMUNAL)]',
681 : 'EMRPESA SARMIENTO S.R.L.[CRUZ DEL EJE]',
682 : 'INTERCORDOBA S.A.[MENDIOLAZA]',
683 : 'INTERCORDOBA S.A.[SALSIPUEDES]',
684 : 'INTERCORDOBA S.A.[UNQUILLO]',
685 : 'MUNICIPALIDAD DE BELL VILLE',
686 : 'SOCSA S.R.L.[VILLA DOLORES]',
687 : 'LATITUD CERO S.A. (SIERRAS SUR)',
688 : 'CAR-COR S.R.L.[VILLA CARLOS PAZ]',
}

#verifico que me haya quedado el diccionario
type(Diccionario_Empresas)

dict

In [26]:
#mapeo el diccionario
base_inter['Nombre_Empresas'] = base_inter['ID_EMPRESA'].map(Diccionario_Empresas)
base_inter.head(5)

,ID_AUTORIZACION,CUIL,ID_ORIGEN,ID_DESTINO,ID_EMPRESA,INICIO_FRANJA_HORARIA,FINAL_FRANJA_HORARIA,CANT_USO_X_DIA,CANT_USO_X_MES,CANT_USO_X_DIA_SEM,CANT_USO_X_SEMANA,ID_TIPO_SOLICITANTE,VIAJA_FERIADO_SN,FEC_DESDE,FEC_HASTA,ID_NIVEL_EDUCATIVO,EXCEP,ID_DISTANCIA,ID_MOVIMIENTO,FEC_MOVIMIENTO,ID_ORIGEN.1,ID_DESTINO.1,ID_PROGRAMA,UID_TARJETA,LATITUD,LONGITUD,NRO_INTERNO,ID_ADMINISTRADORA,VALIDADO_SN,VALORIZAR_SN,FEC_LIQUIDACION,LIQUIDADO_SN,CUIT_TRANSPORTISTA,PRECIO_GOBIERNO,PRECIO_VALORIZADO,ID_RESULTADO,ID_EMPRESA.1,FEC_INGRESO,MONTO_ULTIMO,PROGRAMA,DIA_MOVIMIENTO,ANO_MES_MOVIMIENTO,ANO_MOVIMIENTO,TIPO_INTER,TIPO_INTER_TRAMOS,FILTER,Nombre_Empresas
0,4328075,27027567237,1092.0,1343.0,673,00:00,23:59,2,0,0,0,8,S,12/09/2019 09:13:41 a.m.,19/09/2019,NaN,NaN,6.0,154828600,11/11/2022 01:14:08 p.m.,1092,1343,3,4987AD76,30.980131,64.094316,690,1,S,S,NaN,NaN,30678714344,NaN,"214,9125",3,673,12/11/2022 12:00:51 a.m.,"9891,92",BAM,11/11/2022,11/2022,2022,Interurbano más de 100km,Interurbano menos de 100km,nan,EMPRENDIMIENTOS S.R.L. (FONO BUS)[INTER]
1,4695462,20084531155,1248.0,1.0,677,00:00,23:59,2,0,0,0,8,S,26/10/2019 12:18:32 p.m.,02/11/2019,NaN,NaN,6.0,154906161,14/11/2022 05:02:28 p.m.,1,201,3,69DABF76,31.425068,64.172750,5100,1,S,S,NaN,NaN,30707788832,NaN,"287,6475",3,677,15/11/2022 12:46:00 a.m.,"8992,37",BAM,14/11/2022,11/2022,2022,Interurbano más de 100km,Interurbano menos de 100km,nan,ERSA URBANO S.A.[Inter]
2,4852194,27100483667,1.0,1445.0,677,00:00,23:59,2,0,0,0,8,S,13/11/2019 12:30:10 p.m.,20/11/2019,NaN,NaN,6.0,155270018,21/11/2022 07:22:30 p.m.,1,1445,3,29A4C076,31.350008,64.176090,5239,1,S,S,NaN,NaN,30707788832,NaN,"168,8625",3,677,22/11/2022 12:14:06 a.m.,"13131,31",BAM,21/11/2022,11/2022,2022,Interurbano más de 100km,Interurbano menos de 100km,nan,ERSA URBANO S.A.[Inter]
3,4958275,20079767213,1248.0,1.0,677,00:00,23:59,2,0,0,0,8,S,27/11/2019 11:08:20 a.m.,04/12/2019,NaN,NaN,6.0,154518780,09/11/2022 06:52:46 p.m.,1,1248,3,99CBEB33,31.424975,64.172483,5117,1,S,S,NaN,NaN,30707788832,NaN,"287,6475",3,677,10/11/2022 02:10:54 p.m.,"8757,48",BAM,09/11/2022,11/2022,2022,Interurbano más de 100km,Interurbano menos de 100km,nan,ERSA URBANO S.A.[Inter]
4,5208916,27101141492,6103.0,1498.0,677,00:00,23:59,2,0,0,0,8,S,27/12/2019 01:35:00 p.m.,03/01/2020,NaN,NaN,6.0,155623546,26/11/2022 03:26:31 p.m.,1498,1092,3,79DB7C1D,31.274548,64.169763,5223,1,S,N,NaN,NaN,30707788832,NaN,NaN,3,677,27/11/2022 12:12:22 a.m.,"10100,00",BAM,26/11/2022,11/2022,2022,Interurbano más de 100km,Interurbano menos de 100km,nan,ERSA URBANO S.A.[Inter]


Id_RESULTADO 3 = MOVIMIENTO RECHAZADO

Verifico que sea que lo rechace. Es decir, la persona tiene autorizacion para UM y se quiso tomar un interurbano 

## 1° Etapa: Análisis de Comportamiento Usuarios Urbano Interior

#### Cantidad de Movimientos Totales por Año-Mes y Programa

Para obtener la cantidad de movimientos totales por mes y programa, se agrupa a la tabla por la columna "ANO_MES_MOVIMIENTO" y "PROGRAMA" y se cuenta la cantidad de "ID_MOVIMIENTO". En este caso no es necesario indicar que cuente movimientos únicos, ya que por su naturaleza la columna tiene todos códigos únicos que que representan cada movimiento realizado por los usuarios.

In [27]:
mov_tot = base_inter.groupby(['PROGRAMA', 'TIPO_INTER']).agg({'ID_MOVIMIENTO':'count'}).reset_index()
mov_tot.rename({'ID_MOVIMIENTO':'CANT_MOVIMIENTOS'},axis=1)
mov_tot['TIPO_SERVICIO'] = 'Interurbano'
mov_tot

,PROGRAMA,TIPO_INTER,ID_MOVIMIENTO,TIPO_SERVICIO
0,BAM,Interurbano más de 100km,33332,Interurbano
1,BOS,Interurbano más de 100km,19530,Interurbano


Con el código que sigue, generamos una nueva hoja en el archivo excel creado al comienzo donde se va a guardar la tabla "mov_tot". En el parámetro sheet_name se puede elegir el nombre que se le colocará a la hoja del Excel.

In [28]:
mov_tot.to_excel(writer, sheet_name= 'MOV_TOT_MES')

#### Cantidad de Movimientos Totales por Año-Mes por Cat. y Programa

Acá queremos obtener la cantidad de movimientos totales por mes teniendo en cuenta el Programa, el Tipo de Solicitante y el Nivel Educativo al que pertenece (si corresponde). Agrupamos entonces por las variables "ANO_MES_MOVIMIENTO", "PROGRAMA","ID_TIPO_SOLICITANTE" y "ID_NIVEL_EDUCATIVO" y contamos la cantidad de movimientos. Después de obtener el resultado, necesitamos unir esta tabla con la tabla que contiene las referencia de Tipo Solicitante y Nivel Educativo. Para eso, se utiliza la función merge. Como solo el programa BEG cuenta con nivel educativo, necesitamos rellenar esa columna para el resto de los programas ya que la función groupby no muestra las filas con nulls.

In [29]:
mov_cat = base_inter
mov_cat['ID_NIVEL_EDUCATIVO'] = mov_cat['ID_NIVEL_EDUCATIVO'].fillna(0)
mov_cat.head(3)

,ID_AUTORIZACION,CUIL,ID_ORIGEN,ID_DESTINO,ID_EMPRESA,INICIO_FRANJA_HORARIA,FINAL_FRANJA_HORARIA,CANT_USO_X_DIA,CANT_USO_X_MES,CANT_USO_X_DIA_SEM,CANT_USO_X_SEMANA,ID_TIPO_SOLICITANTE,VIAJA_FERIADO_SN,FEC_DESDE,FEC_HASTA,ID_NIVEL_EDUCATIVO,EXCEP,ID_DISTANCIA,ID_MOVIMIENTO,FEC_MOVIMIENTO,ID_ORIGEN.1,ID_DESTINO.1,ID_PROGRAMA,UID_TARJETA,LATITUD,LONGITUD,NRO_INTERNO,ID_ADMINISTRADORA,VALIDADO_SN,VALORIZAR_SN,FEC_LIQUIDACION,LIQUIDADO_SN,CUIT_TRANSPORTISTA,PRECIO_GOBIERNO,PRECIO_VALORIZADO,ID_RESULTADO,ID_EMPRESA.1,FEC_INGRESO,MONTO_ULTIMO,PROGRAMA,DIA_MOVIMIENTO,ANO_MES_MOVIMIENTO,ANO_MOVIMIENTO,TIPO_INTER,TIPO_INTER_TRAMOS,FILTER,Nombre_Empresas
0,4328075,27027567237,1092.0,1343.0,673,00:00,23:59,2,0,0,0,8,S,12/09/2019 09:13:41 a.m.,19/09/2019,0.0,NaN,6.0,154828600,11/11/2022 01:14:08 p.m.,1092,1343,3,4987AD76,30.980131,64.094316,690,1,S,S,NaN,NaN,30678714344,NaN,"214,9125",3,673,12/11/2022 12:00:51 a.m.,"9891,92",BAM,11/11/2022,11/2022,2022,Interurbano más de 100km,Interurbano menos de 100km,nan,EMPRENDIMIENTOS S.R.L. (FONO BUS)[INTER]
1,4695462,20084531155,1248.0,1.0,677,00:00,23:59,2,0,0,0,8,S,26/10/2019 12:18:32 p.m.,02/11/2019,0.0,NaN,6.0,154906161,14/11/2022 05:02:28 p.m.,1,201,3,69DABF76,31.425068,64.172750,5100,1,S,S,NaN,NaN,30707788832,NaN,"287,6475",3,677,15/11/2022 12:46:00 a.m.,"8992,37",BAM,14/11/2022,11/2022,2022,Interurbano más de 100km,Interurbano menos de 100km,nan,ERSA URBANO S.A.[Inter]
2,4852194,27100483667,1.0,1445.0,677,00:00,23:59,2,0,0,0,8,S,13/11/2019 12:30:10 p.m.,20/11/2019,0.0,NaN,6.0,155270018,21/11/2022 07:22:30 p.m.,1,1445,3,29A4C076,31.350008,64.176090,5239,1,S,S,NaN,NaN,30707788832,NaN,"168,8625",3,677,22/11/2022 12:14:06 a.m.,"13131,31",BAM,21/11/2022,11/2022,2022,Interurbano más de 100km,Interurbano menos de 100km,nan,ERSA URBANO S.A.[Inter]


In [30]:
mov_cat = mov_cat.groupby(['PROGRAMA','TIPO_INTER','ID_TIPO_SOLICITANTE', 'ID_NIVEL_EDUCATIVO']).agg({'ID_MOVIMIENTO':'count'}).reset_index()

mov_cat = mov_cat.merge(tipo_solicitante, on='ID_TIPO_SOLICITANTE', how='left')
mov_cat = mov_cat.merge(nivel_educativo, on='ID_NIVEL_EDUCATIVO', how='left')

mov_cat = mov_cat.rename({'ID_MOVIMIENTO':'CANT_MOVIMIENTOS_MES'})

mov_cat = mov_cat.drop(columns = ['ID_TIPO_SOLICITANTE', 'ID_NIVEL_EDUCATIVO'])#.set_index(['ANO_MES_MOVIMIENTO','PROGRAMA','TIPO_SOLICITANTE','NIVEL_EDUCATIVO'])

#pd.set_option('display.max_rows', None)
mov_cat.rename({'ID_MOVIMIENTO':'CANT_MOVIMIENTOS'},axis=1)
mov_cat['TIPO_SERVICIO'] = 'Interurbano'

In [31]:
mov_cat.to_excel(writer, sheet_name= 'MOV_CAT_MES')

#### Cantidad de Movimientos por Empresa por Mes y Programa

En este caso, obtenemos la cantidad de movimientos totales por mes teniendo en cuenta la empresa que indica la autorización. Al igual que en el resultado anterior, después de agrupar y obtener el resultado, realizamos un merge con la tabla de referencias para poder obtener el nombre de cada empresa.

In [32]:
mov_emp = base_inter.groupby(['PROGRAMA','TIPO_INTER', 'ID_EMPRESA.1'])['ID_MOVIMIENTO'].count().reset_index()
mov_emp = mov_emp.merge(empresas, on='ID_EMPRESA.1', how='left')
mov_emp = mov_emp.rename({'ID_MOVIMIENTO':'CANT_MOVIMIENTOS'}, axis=1)
mov_emp = mov_emp.drop(columns=['ID_TIPO_EMPRESA', 'GRUPO_COMERCIAL', 'TIPO_EMPRESA'])
mov_emp['TIPO_SERVICIO'] = 'Interurbano'


In [33]:
mov_emp.head(5)

,PROGRAMA,TIPO_INTER,ID_EMPRESA.1,CANT_MOVIMIENTOS,ID_EMPRESA,RAZON_SOC,CUIL,TIPO_SERVICIO
0,BAM,Interurbano más de 100km,1,1715,1,COATA S.A.,30-54624472-1,Interurbano
1,BAM,Interurbano más de 100km,2,2636,2,COTA LA CALERA LTDA.,30-54633555-7,Interurbano
2,BAM,Interurbano más de 100km,4,148,4,COLONIA TIROLESA S.R.L.,30-59873589-8,Interurbano
3,BAM,Interurbano más de 100km,5,2134,5,EMPRESA CORDOBA S.R.L.,30-54622193-4,Interurbano
4,BAM,Interurbano más de 100km,6,506,6,EL PORVENIR S.R.L.,30-54643075-4,Interurbano


#### Check Empresas sin Razon Social

Aparecen nulls en la columna RAZON_SOC hay que agregar las empresas faltantes en el archivo de Referencias.

In [34]:
check = mov_emp
check = check[check['RAZON_SOC'].isnull()]
check['ID_EMPRESA.1'].unique()

array([], dtype=int64)

In [35]:
mov_emp.to_excel(writer, sheet_name= 'MOV_EMP_MES')

#### Cantidad Beneficiarios Activos por Año-Mes

Para obtener la cantidad de Beneficiarios Activos por mes, agrupamos teniendo en cuenta CUILs únicos. Como un mismo beneficiario realiza más de un movimiento por mes, necesitamos contar solo una aparición de cada CUIL.

In [36]:
beneficiarios_mes = base_inter.groupby(['PROGRAMA', 'TIPO_INTER']).agg({'CUIL':'nunique'}).reset_index()
beneficiarios_mes.rename({'CUIL':'CANT_BENEFICIARIOS'},axis=1)
beneficiarios_mes['TIPO_SERVICIO'] = 'Interurbano'
beneficiarios_mes

,PROGRAMA,TIPO_INTER,CUIL,TIPO_SERVICIO
0,BAM,Interurbano más de 100km,15703,Interurbano
1,BOS,Interurbano más de 100km,737,Interurbano


In [37]:
beneficiarios_mes.to_excel(writer, sheet_name= 'BENEFICIARIOS_MES')

#### Cantidad Beneficiarios Activos por Mes por Cat

En este punto, vamos a obtener la cantidad de Beneficiarios Activos por mes teniendo en cuenta el tipo de solicitante y el nivel educativo al que pertenecen. El número obtenido puede no coincidir con el resultado anterior ya que se contarán CUILs únicos dentro de cada categoría y un mismo CUIL puede estar incluido en la categoría Docente y Alumno por ejemplo. El resultado debe ser igual o mayor al obtenido en el paso anterior.

In [38]:
beneficiarios_cat = base_inter.groupby(['PROGRAMA','TIPO_INTER','ID_TIPO_SOLICITANTE', 'ID_NIVEL_EDUCATIVO']).agg({'CUIL':'nunique'}).reset_index()
beneficiarios_cat = beneficiarios_cat.merge(tipo_solicitante, on='ID_TIPO_SOLICITANTE', how='left')
beneficiarios_cat = beneficiarios_cat.merge(nivel_educativo, on='ID_NIVEL_EDUCATIVO', how='left')

beneficiarios_cat = beneficiarios_cat.rename({'CUIL':'CANT_BENEFICIARIOS'}, axis=1)
beneficiarios_cat = beneficiarios_cat.drop(columns = ['ID_TIPO_SOLICITANTE', 'ID_NIVEL_EDUCATIVO'])#.set_index(['ANO_MOVIMIENTO','NIVEL_EDUCATIVO', 'TIPO_SOLICITANTE'])
beneficiarios_cat['TIPO_SERVICIO'] = 'Interurbano'

In [39]:
beneficiarios_cat.to_excel(writer, sheet_name= 'BENEFICIARIOS_CAT')

#### Cantidad Beneficios Activos por Año-Mes

En este paso, lo que se busca es obtener la cantidad de Beneficios Activos Totales por Mes. La cantidad de Beneficios y Beneficiarios suele diferir ya que un beneficiario puede obtener más de un beneficio. Se agrupa teniendo en cuenta apariciones únicas de la variable 'ID_AUTORIZACIÓN' que es la que identifica a cada beneficio entregado. 

In [40]:
beneficios_mes = base_inter.groupby(['PROGRAMA', 'TIPO_INTER']).agg({'ID_AUTORIZACION':'nunique'}).reset_index()
beneficios_mes.rename({'ID_AUTORIZACION':'CANT_BENEFICIOS'},axis=1)
beneficios_mes['TIPO_SERVICIO'] = 'Interurbano'

In [41]:
beneficios_mes.to_excel(writer, sheet_name= 'BENEFICIOS_MES')

#### Cantidad de Beneficios Activos por Programa, Año-Mes y Categoría

Al resultado anterior se lo visualiza acá abierto por Programa, Tipo de Solicitante y Nivel Educativo. En este caso, al tratarse de beneficios y no beneficiarios, el resultado cerrado del punto anterior debe coincidir con el resultado obtenido aquí.

In [42]:
beneficios_cat = base_inter.groupby(['PROGRAMA','TIPO_INTER','ID_TIPO_SOLICITANTE', 'ID_NIVEL_EDUCATIVO']).agg({'ID_AUTORIZACION':'nunique'})
beneficios_cat = beneficios_cat.reset_index()
beneficios_cat = beneficios_cat.merge(tipo_solicitante, on='ID_TIPO_SOLICITANTE', how='left')
beneficios_cat = beneficios_cat.merge(nivel_educativo, on='ID_NIVEL_EDUCATIVO', how='left')

beneficios_cat = beneficios_cat.drop(columns = ['ID_TIPO_SOLICITANTE', 'ID_NIVEL_EDUCATIVO'])#.set_index(['ANO_MOVIMIENTO', 'NIVEL_EDUCATIVO', 'TIPO_SOLICITANTE'])
beneficios_cat.rename({'ID_AUTORIZACION':'CANT_BENEFICIOS'},axis=1)
beneficios_cat['TIPO_SERVICIO'] = 'Interurbano'

In [43]:
beneficios_cat.to_excel(writer, sheet_name= 'BENEFICIOS_CAT')

#### Beneficiarios con Más de un Beneficio en Mismo Servicio

En este punto, se busca saber qué cantidad de beneficios tiene cada beneficiario. Para ello, en un primer paso se agrupa a la tabla por CUIL teniendo en cuenta ID_AUTORIZACION únicos. Esto nos mostrará cuantas autorizaciones tiene cada uno de los CUILs que aparece en la tabla. Luego se reagrupa la tabla en función del resultado anterior, contando la cantidad de CUIL únicos por cantidad de autorizaciones.

In [44]:
multibeneficios = base_inter[base_inter['PROGRAMA']=='BEG']
multibeneficios = base_inter.groupby(['PROGRAMA','TIPO_INTER','CUIL']).agg({'ID_AUTORIZACION':'nunique'}).reset_index()
multibeneficios = multibeneficios.rename({'ID_AUTORIZACION':'CANT_AUTORIZACIONES'},axis=1)
multibeneficios = multibeneficios.groupby(['PROGRAMA','TIPO_INTER','CANT_AUTORIZACIONES']).agg({'CUIL':'count'}).reset_index()
multibeneficios.rename({'CUIL':'BENEFICIARIOS'},axis=1)
multibeneficios['TIPO_SERVICIO'] = 'Interurbano'
multibeneficios

,PROGRAMA,TIPO_INTER,CANT_AUTORIZACIONES,CUIL,TIPO_SERVICIO
0,BAM,Interurbano más de 100km,1,9687,Interurbano
1,BAM,Interurbano más de 100km,2,4804,Interurbano
2,BAM,Interurbano más de 100km,3,797,Interurbano
3,BAM,Interurbano más de 100km,4,320,Interurbano
4,BAM,Interurbano más de 100km,5,53,Interurbano
5,BAM,Interurbano más de 100km,6,26,Interurbano
6,BAM,Interurbano más de 100km,7,10,Interurbano
7,BAM,Interurbano más de 100km,8,6,Interurbano
8,BOS,Interurbano más de 100km,1,733,Interurbano
9,BOS,Interurbano más de 100km,2,4,Interurbano


In [45]:
multibeneficios.to_excel(writer, sheet_name= 'MULTIBENEFICIOS')

### Tasa Utilización por Mes por Autorizacion

La lógica para el cálculo de la Tasa de Utilización por Mes varia según el programa. Este cálculo se hace por separado para cada uno de los 3 programas para finalmente obtener un resultado agrupado

### 1. Tasa Utilización BEG

#### 1.1 Filtra Base para BEG

In [46]:
base_beg = base_inter.copy()
base_beg = base_beg[base_beg['PROGRAMA']=='BEG']

#### 1.2 Calcula Variable Cantidad de Usos Autorizados

En el caso de los boletos BEG Urbano Interior, la variable de interés que nos permite saber los días autorizados para cada beneficiario es "CANT_USO_X_DIA_SEM" por lo que nos interesa analizarla en profundidad. Para empezar, necesitamos que no existan casos nulos en esta columna para poder realizar distintas operaciones. Por eso, rellenamos los valores nulos que puedan aparecer con 0.

In [47]:
base_beg['CANT_USO_X_DIA_SEM'] = base_beg['CANT_USO_X_DIA_SEM'].fillna(0) 

#### 1.2.1 Crea Columna para Cada Día de la Semana con Boletos Autorizados

La variable "CANT_USO_X_DIA_SEM" indica con un valor (que indica la cantidad de boletos autorizados) si es un día en el que beneficiario tiene autorización para utilizar la tarjeta y con 0 en caso contrario. Para saber cuántos boletos tiene autorizado en total, tenemos que sumar los digitos de esta columna. Para eso, usamos la función digitize que separa cada uno de los dígitos en una columna separada, facilitando el cálculo.

In [48]:
def digit(x, n):
    """Return the n-th digit of integer in base 10"""
    return (x // 10**n) % 10

def digitize(df, key, n):
    """Extract n less significant digits from an integer in base 10"""
    for i in range(n):
        df['x%d' % i] = digit(df[key], n-i-1) 

In [49]:
digitize(base_beg, 'CANT_USO_X_DIA_SEM', 7)

In [50]:
dict = {'x0':'Monday',
        'x1':'Tuesday',
        'x2':'Wednesday',
        'x3':'Thursday',
        'x4':'Friday',
        'x5':'Saturday',
        'x6':'Sunday'
       }
base_beg.rename(columns=dict, inplace=True)

In [51]:
cols_to_sum = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
base_beg['CANT_AUTO_X_SEM'] = base_beg[cols_to_sum].sum(axis=1)

In [52]:
beg_menos100 = base_beg.copy()
beg_menos100 = base_beg[base_beg['TIPO_INTER']=='Interurbano menos de 100km']
beg_mas100 = base_beg.copy()
beg_mas100 = base_beg[base_beg['TIPO_INTER']=='Interurbano mas de 100km']

 #### 1.2 Calcula Tasa Uso BEG Interurbano menos de 100km

En este punto se obtiene la tasa de utilización mensual de los boletos por Tipo de Solicitante y Nivel Educativo. Para ello se siguen los pasos a continuación:
   1. Se agrupa la tabla por Mes, Tipo de Solicitante, Nivel Educativo, Autorización y Cantidad Autorizada por Semana.
   2. Se modifica el formato de la variable Mes para que quede expresada con el formato de date lo que permitirá seguir trabajando.
   3. Se crea una variable auxiliar "ULT_DIA_MES" que nos indica el ultimo día del mes correspondiente a la variables "ANO_MES_MOVIMIENTO"
   4. Con la función np.busday_count() se obtiene la cantidad de días hábiles en el mes. La función necesita tener indicado un comienzo y un final del intervalo, razón por la cual se creó la variable auxiliar anterior.
   5. Luego de obtener la cantidad de días hábiles en el mes, se divide el resultado por 5 para obtener la cantidad de semanas promedio de ese mes ("CANT_SEM_MES").
   6. Se obtiene la cantidad total de boletos autorizados en el mes multiplicando la cantidad de boletos autorizados por semana por la cantidad de semanas en el mes.
   7. Finalmente, la tasa de uso es obtenida como el cociente entre la cantidad de movimientos utilizados y los autorizados.

In [53]:
tasa_beg_menos100 = beg_menos100.groupby(['ANO_MES_MOVIMIENTO','PROGRAMA','TIPO_INTER','ID_TIPO_SOLICITANTE', 'ID_NIVEL_EDUCATIVO','ID_AUTORIZACION', 'CANT_AUTO_X_SEM'])['ID_MOVIMIENTO'].count().reset_index()
tasa_beg_menos100['ANO_MES_MOVIMIENTO_aux'] = pd.to_datetime(tasa_beg_menos100['ANO_MES_MOVIMIENTO'])

tasa_beg_menos100['ULT_DIA_MES'] = tasa_beg_menos100['ANO_MES_MOVIMIENTO_aux'] + pd.offsets.MonthEnd(1)

tasa_beg_menos100['DIAS_HABILES'] = np.busday_count(tasa_beg_menos100['ANO_MES_MOVIMIENTO_aux'].values.astype('datetime64[D]'),tasa_beg_menos100['ULT_DIA_MES'].values.astype('datetime64[D]'))

tasa_beg_menos100['CANT_SEM_MES'] = tasa_beg_menos100['DIAS_HABILES']/5

tasa_beg_menos100['CANT_AUTO_MES'] = tasa_beg_menos100['CANT_SEM_MES']*tasa_beg_menos100['CANT_AUTO_X_SEM']
tasa_beg_menos100['TASA_USO'] = tasa_beg_menos100['ID_MOVIMIENTO']/tasa_beg_menos100['CANT_AUTO_MES']

tasa_beg_menos100 = tasa_beg_menos100.merge(tipo_solicitante, on='ID_TIPO_SOLICITANTE', how='left')
tasa_beg_menos100 = tasa_beg_menos100.merge(nivel_educativo, on='ID_NIVEL_EDUCATIVO', how='left')

tasa_beg_menos100 = tasa_beg_menos100.drop(columns = ['ID_TIPO_SOLICITANTE', 'ID_NIVEL_EDUCATIVO'])
tasa_beg_menos100 = tasa_beg_menos100[['ANO_MES_MOVIMIENTO', 'PROGRAMA','TIPO_INTER','ID_AUTORIZACION', 'TIPO_SOLICITANTE', 'NIVEL_EDUCATIVO', 'TASA_USO']]
tasa_beg_menos100.head(5)

,ANO_MES_MOVIMIENTO,PROGRAMA,TIPO_INTER,ID_AUTORIZACION,TIPO_SOLICITANTE,NIVEL_EDUCATIVO,TASA_USO


 #### 1.3 Calcula Tasa Uso BEG Interurbano más de 100km

En el caso de los Interurbanos más de 100km, las autorizaciones son mensuales. Tenemos que utilizar la variable "CANT_USO_X_MES".

In [54]:
pd.value_counts (base_beg['TIPO_INTER'])

Series([], Name: TIPO_INTER, dtype: int64)

In [55]:
beg_mas100 = base_beg.copy()
beg_mas100 = base_beg[base_beg['TIPO_INTER']=='Interurbano más de 100km']

In [56]:
tasa_beg_mas100 = beg_mas100.groupby(['ANO_MES_MOVIMIENTO','PROGRAMA','TIPO_INTER','ID_TIPO_SOLICITANTE', 'ID_NIVEL_EDUCATIVO','ID_AUTORIZACION','CANT_USO_X_MES'])['ID_MOVIMIENTO'].count().reset_index()
tasa_beg_mas100 = tasa_beg_mas100.rename({'ID_MOVIMIENTO':'CANT_MOVIMIENTOS'},axis=1)
tasa_beg_mas100['TASA_USO'] = tasa_beg_mas100['CANT_MOVIMIENTOS']/tasa_beg_mas100['CANT_USO_X_MES']

tasa_beg_mas100 = tasa_beg_mas100.merge(tipo_solicitante, on='ID_TIPO_SOLICITANTE', how='left')
tasa_beg_mas100 = tasa_beg_mas100.merge(nivel_educativo, on='ID_NIVEL_EDUCATIVO', how='left')

tasa_beg_mas100 = tasa_beg_mas100.drop(columns = ['ID_TIPO_SOLICITANTE', 'ID_NIVEL_EDUCATIVO'])
tasa_beg_mas100 = tasa_beg_mas100[['ANO_MES_MOVIMIENTO', 'PROGRAMA','TIPO_INTER','ID_AUTORIZACION', 'TIPO_SOLICITANTE', 'NIVEL_EDUCATIVO', 'TASA_USO']]
tasa_beg_mas100.head(5)

,ANO_MES_MOVIMIENTO,PROGRAMA,TIPO_INTER,ID_AUTORIZACION,TIPO_SOLICITANTE,NIVEL_EDUCATIVO,TASA_USO


In [57]:
tasa_beg_mas100.shape

(0, 7)

### 2. Tasa Utilización BOS

#### 2.1 Filtra Base para BOS

In [58]:
#base_bos = base_inter.copy()
#base_bos = base_bos[base_bos['PROGRAMA']=='BOS']
#base_bos

#### 2.2 Calcula Variable Cantidad de Usos Autorizados

En el caso de los boletos BOS Interurbano, la variable de interés que nos permite saber los días autorizados para cada beneficiario es "CANT_USO_X_DIA_SEM" por lo que nos interesa analizarla en profundidad. Para empezar, necesitamos que no existan casos nulos en esta columna para poder realizar distintas operaciones. Por eso, rellenamos los valores nulos que puedan aparecer con 0.

In [59]:
#base_bos['CANT_USO_X_DIA_SEM'] = base_bos['CANT_USO_X_DIA_SEM'].fillna(0) 
#base_bos

#### 2.2.1 Crea Columna para Cada Día de la Semana con Boletos Autorizados

In [60]:
#digitize(base_bos, 'CANT_USO_X_DIA_SEM', 7)
#base_bos

In [61]:
#dict = {'x0':'Monday',
 #       'x1':'Tuesday',
  #      'x2':'Wednesday',
   #     'x3':'Thursday',
    #    'x4':'Friday',
     #  'x5':'Saturday',
      #  'x6':'Sunday' }
#base_bos.rename(columns=dict, inplace=True)
#base_bos

In [62]:
#cols_to_sum = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
#base_bos['CANT_AUTO_X_SEM'] = base_bos[cols_to_sum].sum(axis=1)
#base_bos

 #### 2.2 Calcula Tasa Uso BOS

En este punto se obtiene la tasa de utilización mensual de los boletos por Tipo de Solicitante. Para ello se siguen los pasos a continuación:
   1. Se agrupa la tabla por Mes, Tipo de Solicitante, Autorización y Cantidad Autorizada por Semana.
   2. Se modifica el formato de la variable Mes para que quede expresada con el formato de date lo que permitirá seguir trabajando.
   3. Se crea una variable auxiliar "ULT_DIA_MES" que nos indica el ultimo día del mes correspondiente a la variables "ANO_MES_MOVIMIENTO"
   4. Con la función np.busday_count() se obtiene la cantidad de días hábiles en el mes. La función necesita tener indicado un comienzo y un final del intervalo, razón por la cual se creó la variable auxiliar anterior.
   5. Luego de obtener la cantidad de días hábiles en el mes, se divide el resultado por 5 para obtener la cantidad de semanas promedio de ese mes ("CANT_SEM_MES").
   6. Se obtiene la cantidad total de boletos autorizados en el mes multiplicando la cantidad de boletos autorizados por semana por la cantidad de semanas en el mes.
   7. Finalmente, la tasa de uso es obtenida como el cociente entre la cantidad de movimientos utilizados y los autorizados.

In [63]:
#tasa_bos = base_bos.groupby(['ANO_MES_MOVIMIENTO','PROGRAMA','TIPO_INTER','ID_TIPO_SOLICITANTE','ID_NIVEL_EDUCATIVO','ID_AUTORIZACION', 'CANT_AUTO_X_SEM'])['ID_MOVIMIENTO'].count().reset_index()
#tasa_bos['ANO_MES_MOVIMIENTO_aux'] = pd.to_datetime(tasa_bos['ANO_MES_MOVIMIENTO'])
#tasa_bos['ULT_DIA_MES'] = tasa_bos['ANO_MES_MOVIMIENTO_aux'] + pd.offsets.MonthEnd(1)

#tasa_bos['DIAS_HABILES'] = np.busday_count(tasa_bos['ANO_MES_MOVIMIENTO_aux'].values.astype('datetime64[D]'),tasa_bos['ULT_DIA_MES'].values.astype('datetime64[D]'))

#tasa_bos['CANT_SEM_MES'] = tasa_bos['DIAS_HABILES']/5

#tasa_bos['CANT_AUTO_MES'] = tasa_bos['CANT_SEM_MES']*tasa_bos['CANT_AUTO_X_SEM']
#tasa_bos['CANT_AUTO_MES'] = np.where(tasa_bos['CANT_AUTO_MES']==0, 16, tasa_bos['CANT_AUTO_MES'])
#tasa_bos['TASA_USO'] = tasa_bos['ID_MOVIMIENTO']/tasa_bos['CANT_AUTO_MES']

#tasa_bos = tasa_bos.merge(tipo_solicitante, on='ID_TIPO_SOLICITANTE', how='left')
#tasa_bos = tasa_bos.merge(nivel_educativo, on='ID_NIVEL_EDUCATIVO', how='left')

#tasa_bos = tasa_bos.drop(columns = ['ID_TIPO_SOLICITANTE', 'ID_NIVEL_EDUCATIVO'])
#tasa_bos = tasa_bos[['ANO_MES_MOVIMIENTO', 'PROGRAMA','TIPO_INTER','ID_AUTORIZACION', 'TIPO_SOLICITANTE', 'NIVEL_EDUCATIVO', 'TASA_USO']]

#tasa_bos

### 3. Tasa Utilización BAM

#### 3.1 Filtra Base para BAM

base_bam = base_inter.copy()
base_bam = base_bam[base_bam['PROGRAMA']=='BAM']
base_bam['TIPO_SERVICIO']='Interurbano'
base_bam

test = base_bam[(base_bam['ID_AUTORIZACION']==6446622)&(base_bam['ANO_MES_MOVIMIENTO']=='02/2021')]
test

#### 3.2 Une con Tabla Referencias con Montos

base_bam = base_bam.merge(regla_bam, on=['TIPO_SERVICIO', 'ID_DISTANCIA', 'PROGRAMA'], how='left')
base_bam['MONTO_ULTIMO'] = base_bam['MONTO_ULTIMO'].str.replace(',', '.').astype(float) 
base_bam = base_bam[base_bam['MONTO_ULTIMO'].between(base_bam['MONTO_DESDE'],base_bam['MONTO_HASTA'])]
base_bam

#### 3.2 Calcula Tasa Uso BAM

En este punto se obtiene la tasa de utilización mensual de los boletos. Para ello se siguen los pasos a continuación:
   1. Se agrupa la tabla por Mes y Cantidad Autorizada por Mes.
   2. La tasa de uso es obtenida como el cociente entre la cantidad de movimientos utilizados y los autorizados.

tasa_bam = base_bam
tasa_bam = tasa_bam.groupby(['ANO_MES_MOVIMIENTO','PROGRAMA','TIPO_INTER','ID_TIPO_SOLICITANTE', 'ID_NIVEL_EDUCATIVO','ID_AUTORIZACION', 'CANT_BOLETOS'])['ID_MOVIMIENTO'].count().reset_index()
tasa_bam['TASA_USO'] = tasa_bam['ID_MOVIMIENTO']/tasa_bam['CANT_BOLETOS']

tasa_bam
tasa_bam = tasa_bam.merge(tipo_solicitante, on='ID_TIPO_SOLICITANTE', how='left')
tasa_bam = tasa_bam.merge(nivel_educativo, on='ID_NIVEL_EDUCATIVO', how='left')

tasa_bam = tasa_bam.drop(columns = ['ID_TIPO_SOLICITANTE', 'ID_NIVEL_EDUCATIVO'])
tasa_bam = tasa_bam[['ANO_MES_MOVIMIENTO', 'PROGRAMA','TIPO_INTER', 'ID_AUTORIZACION', 'TIPO_SOLICITANTE', 'NIVEL_EDUCATIVO', 'TASA_USO']]

tasa_bam

### 5. Tabla Tasa General

In [64]:
tasa = tasa_beg_menos100.append([tasa_beg_mas100])
tasa['TIPO_SERVICIO'] = 'Interurbano'

C:\Users\Control 2\AppData\Local\Temp\ipykernel_5092\1155556068.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tasa = tasa_beg_menos100.append([tasa_beg_mas100])


In [65]:
tasa.to_excel(writer, sheet_name= 'TASA_TABLA')

#### Promedio Tasa Uso por Mes

En el punto anterior, se obtuvo la tasa de uso de cada autorización por mes. Aquí se calcula la tasa promedio por Tipo de Solicitante y Nivel Educativo.

In [66]:
tasa_prom = tasa.copy()

tasa_prom = tasa_prom.groupby(['ANO_MES_MOVIMIENTO','PROGRAMA','TIPO_INTER','TIPO_SOLICITANTE', 'NIVEL_EDUCATIVO']).agg({'TASA_USO':'mean'})
tasa_prom=tasa_prom.reset_index()
tasa_prom['TIPO_SERVICIO'] = 'Interurbano'
tasa_prom

,ANO_MES_MOVIMIENTO,PROGRAMA,TIPO_INTER,TIPO_SOLICITANTE,NIVEL_EDUCATIVO,TASA_USO,TIPO_SERVICIO


In [67]:
tasa_prom.to_excel(writer, sheet_name= 'TASA_USO_PROM')

#### Intervalos Tasa de Uso Promedio

Se crean intervalos para la Tasa de Uso, de manera de saber la cantidad de beneficios que cae en cada uno de estos intervalos. La función para crear intervalos es pd.cut() y necesita que sean definidos tanto los límites de cada intervalo (bins) como el nombre (labels).

#### Casos por Intervalos

In [68]:
intervalos = tasa.copy()

bins = [0,0.25, 0.50,0.75, 1.2, 99999]
labels = ['<25%','25-50%','50-75%','75-100%','+100%']

intervalos['INTERVALOS'] = pd.cut(intervalos['TASA_USO'], bins=bins, labels=labels)
intervalos['TIPO_SERVICIO'] = 'Interurbano'


In [69]:
intervalos.to_excel(writer, sheet_name= 'INTERVALOS_TABLA')

A partir de la tabla creada en el punto anterior, se agrupan las autorizaciones por intervalo contando la cantidad de casos.

In [70]:
casos_rango = intervalos.copy()
casos_rango = casos_rango.groupby(['ANO_MES_MOVIMIENTO','PROGRAMA','TIPO_INTER','INTERVALOS','TIPO_SOLICITANTE','NIVEL_EDUCATIVO']).agg({'ID_AUTORIZACION':'count'}).reset_index()
casos_rango.reset_index()
casos_rango.rename({'ID_AUTORIZACION':'CANT_CASOS'},axis=1)
casos_rango['TIPO_SERVICIO'] = 'Interurbano'
casos_rango

,ANO_MES_MOVIMIENTO,PROGRAMA,TIPO_INTER,INTERVALOS,TIPO_SOLICITANTE,NIVEL_EDUCATIVO,ID_AUTORIZACION,TIPO_SERVICIO


In [71]:
casos_rango.to_excel(writer, sheet_name= 'INTERVALOS_RANGO')

# Guardar Resultados en Excel

Todas las hojas del excel que fuimos definiendo a lo largo de la notebook, las guardamos en este momento como sigue:

In [72]:
writer.save()